# Exploratory Data Analysis

## Environment and Dataset Setup

In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

In [2]:
# import data from csv
raw = pd.read_csv('data/homes.csv')
raw.head()

,Status,MLS#,Property Type,County,Style,Year Built,Master on Main,Rooms,Kitchen,Interior,...,Other,Sprinkler,Veranda,Water Feature,City,State,Zip,Latitude,Longitude,Result_Addy
0,Active,9078099,Rental,DeKalb,Brick 4 Side Ranch,1954.0,Yes,Family Room,Breakfast Room,Other Master On Main Level,...,0,0,0,0,Avondale Estates,GA,30002,-84.25691,33.770443,"3262 MAJESTIC CIR, AVONDALE EST, GA, 30002"
1,Active,20009151,Rental,Fulton,European,1999.0,Yes,Foyer Keeping Room Laundry Bonus Room Exercise...,NaN,High Ceilings Double Vanity Beamed Ceilings Tw...,...,0,0,0,0,Alpharetta,GA,30004,NaN,NaN,NaN
2,New,20010927,Rental,Fulton,European Traditional,1997.0,No,Exercise Room Family Room Foyer Great Room Kee...,NaN,High Ceilings Two Story Foyer Tile Bath Walk-I...,...,0,0,0,0,Alpharetta,GA,30004,-84.26803,34.137955,"14595 CREEK CLUB DR, ALPHARETTA, GA, 30004"
3,Pending Approval,10005432,Rental,Fulton,Traditional,1990.0,No,Foyer Laundry Family Room Other,Breakfast Area Breakfast Bar Breakfast Room Pa...,Vaulted Ceiling(s) Double Vanity Pulldown Atti...,...,0,0,0,0,Alpharetta,GA,30004,-84.34001,34.075980,"12090 WALLACE WOODS LN, ALPHARETTA, GA, 30004"
4,Active,10009045,Rental,Forsyth,Brick 3 Side Craftsman,2009.0,No,Foyer Laundry Office,Breakfast Area Breakfast Bar Breakfast Room Ki...,Bookcases Tray Ceiling(s) High Ceilings Double...,...,0,0,0,0,Alpharetta,GA,30004,-84.24270,34.125890,"3046 HIGHLAND PASS, ALPHARETTA, GA, 30004"


In [3]:
# dimensionality of dataset
print(f'{len(raw.columns)} columns, {len(raw)} rows')

# how many non-NaN datapoints in each column
notna_cols = raw.notna().sum(axis=0)
#print(notna_cols)

# how many non-NaN datapoints in each row
notna_rows = raw.notna().sum(axis=1)
#print(notna_rows)

# how many full rows and columns (no NaNs)?
print(f'{notna_rows.value_counts()[50]} full rows')
print(f'{notna_cols.value_counts()[33740]} full columns')
print()
print(notna_rows.value_counts()) #outputs in format (num values | num rows)

50 columns, 33740 rows
7591 full rows
32 full columns

50    7591
49    7251
48    6809
47    5587
46    3886
45    1459
44     623
43     372
42      98
41      45
40      19
dtype: int64


## Initial Exploration

In [4]:
# view columns of dataset and their unique values
uniques = pd.DataFrame()
for col in raw.columns:
    u_vals = raw[col].unique()
    uniques[col] = list(u_vals) + ['-']*(len(raw) - len(u_vals))

# view number of unique values for each column
num_uniques = pd.DataFrame()
for col in raw.columns:
    num_uniques[col] = [len(raw[col].unique())]